In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X = train_df.drop(columns=["Survived"])
y = train_df["Survived"]
X_test = test_df.copy()


In [3]:
numeric_features = ["Age", "Fare", "SibSp", "Parch"]
categorical_features = ["Sex", "Pclass", "Embarked"]

preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), numeric_features),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), categorical_features)
])


In [4]:
ensemble_model = VotingClassifier(
    estimators=[
        ("lr", LogisticRegression(max_iter=1000)),
        ("rf", RandomForestClassifier(n_estimators=300, max_depth=6)),
        ("gb", GradientBoostingClassifier()),
        ("svm", SVC(probability=True))
    ],
    voting="soft"
)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

ensemble_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("ensemble", ensemble_model)
])

ensemble_pipeline.fit(X_train, y_train)
val_preds = ensemble_pipeline.predict(X_val)

acc = accuracy_score(y_val, val_preds)

print(f"Ensemble Accuracy: {acc:.4f}")
print(f"Ensemble Accuracy (%): {acc*100:.2f}%")


Ensemble Accuracy: 0.8212
Ensemble Accuracy (%): 82.12%


In [6]:
ensemble_pipeline.fit(X, y)

,steps,"[('preprocessing', ...), ('ensemble', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [7]:


test_preds = ensemble_pipeline.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": test_preds.astype(int)
})

submission.to_csv("submission_ensemble.csv", index=False)

print("Saved submission_ensemble.csv")


Saved submission_ensemble.csv
